In [1]:
import GPy
import pandas as pd
import zipfile
import urllib.request
import time
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from sklearn.model_selection import KFold

from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))

True
9.0.176
GeForce RTX 2080 Ti


In [16]:
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out
def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)


def get_kl_divergence(weights, prior, varpost):
    prior_loglik = prior.loglik(weights)
    
    varpost_loglik = varpost.loglik(weights)
    varpost_lik = varpost_loglik.exp()#exp func: y=e^x
    
    return (varpost_lik*(varpost_loglik - prior_loglik)).sum()


class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        scale = (2/self.input_dim)**0.5
        rho_init = np.log(np.exp((2/self.input_dim)**0.5) - 1)
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
        self.bias_mus = nn.Parameter(torch.Tensor(self.output_dim).uniform_(-0.01, 0.01))
        self.bias_rhos = nn.Parameter(torch.Tensor(self.output_dim).uniform_(-4, -3))
        
    def forward(self, x, sample = True):
        
        if sample:
            # sample gaussian noise for each weight and each bias
            weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())
            bias_epsilons =  Variable(self.bias_mus.data.new(self.bias_mus.size()).normal_())
            
            # calculate the weight and bias stds from the rho parameters
            weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
            bias_stds = torch.log(1 + torch.exp(self.bias_rhos))
            
            # calculate samples from the posterior from the sampled noise and mus/stds
            weight_sample = self.weight_mus + weight_epsilons*weight_stds
            bias_sample = self.bias_mus + bias_epsilons*bias_stds
            
            #torch.cuda.synchronize()
            output = torch.mm(x, weight_sample) + bias_sample
            
            # computing the KL loss term
            prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
            KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
            KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
            KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
            prior_cov, varpost_cov = self.prior.sigma**2, bias_stds**2
            KL_loss = KL_loss + 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*bias_stds.numel()
            KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
            KL_loss = KL_loss + 0.5*((self.bias_mus - self.prior.mu)**2/prior_cov).sum()
            
            return output, KL_loss
        
        else:
            output = torch.mm(x, self.weight_mus) + self.bias_mus
            return output, KL_loss
        
    def sample_layer(self, no_samples):
        all_samples = []
        for i in range(no_samples):
            # sample gaussian noise for each weight and each bias
            weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())
            
            # calculate the weight and bias stds from the rho parameters
            weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
            
            # calculate samples from the posterior from the sampled noise and mus/stds
            weight_sample = self.weight_mus + weight_epsilons*weight_stds
            
            all_samples += weight_sample.view(-1).cpu().data.numpy().tolist()
            
        return all_samples
class BBP_Heteroscedastic_Model(nn.Module):
    def __init__(self, input_dim, output_dim, num_units):
        super(BBP_Heteroscedastic_Model, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # network with two hidden and one output layer
        self.layer1 = BayesLinear_Normalq(input_dim, num_units, gaussian(0, 1))
        self.layer2 = BayesLinear_Normalq(num_units, 2*output_dim, gaussian(0, 1))
        
        # activation to be used between hidden layers
        #self.activation = nn.ReLU(inplace = True)
    
    def forward(self, x):
        
        KL_loss_total = 0
        x = x.view(-1, self.input_dim)
        x, KL_loss = self.layer1(x)
        KL_loss_total = KL_loss_total + KL_loss
        #x = self.activation(x)
        
        x, KL_loss = self.layer2(x)
        KL_loss_total = KL_loss_total + KL_loss
        
        return x, KL_loss_total
class BBP_Heteroscedastic_Model_Wrapper:
    def __init__(self, network, learn_rate, batch_size, no_batches):
        
        self.learn_rate = learn_rate
        self.batch_size = batch_size
        self.no_batches = no_batches
        
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = log_gaussian_loss
    
    def fit(self, x, y, no_samples):
        
        x, y = to_variable(var=(x, y), cuda=True)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        fit_loss_total = 0
        
        for i in range(no_samples):
            output, KL_loss_total = self.network(x)
            
            # calculate fit loss based on mean and standard deviation of output
            fit_loss = self.loss_func(output[:, :1], y, output[:, 1:].exp(), 1) 
            fit_loss_total = fit_loss_total + fit_loss
        
        KL_loss_total = KL_loss_total/self.no_batches
        total_loss = (fit_loss_total + KL_loss_total)/(no_samples*x.shape[0])
        total_loss.backward()
        self.optimizer.step()

        return fit_loss_total/no_samples, KL_loss_total
    
    def get_loss_and_rmse(self, x, y, no_samples):
        x, y = to_variable(var=(x, y), cuda=True)
        
        means, stds = [], []
        for i in range(no_samples):
            output, KL_loss_total = self.network(x)
            means.append(output[:, :1, None])
            stds.append(output[:, 1:, None].exp())
            
        means, stds = torch.cat(means, 2), torch.cat(stds, 2)
        mean = means.mean(dim=2)
        std = (means.var(dim=2) + stds.mean(dim=2)**2)**0.5
            
        # calculate fit loss based on mean and standard deviation of output
        logliks = self.loss_func(output[:, :1], y, output[:, 1:].exp(), 1, sum_reduce=False)
        rmse = float((((mean - y)**2).mean()**0.5).cpu().data)

        return logliks, rmse
class BBP_Heteroscedastic_Model_eICU(nn.Module):
    def __init__(self, input_dim, output_dim, num_units):
        super(BBP_Heteroscedastic_Model_eICU, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # network with two hidden and one output layer
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 1))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 1))
        self.layer3 = BayesLinear_Normalq(num_units[1], 2*output_dim, gaussian(0, 1))
        
        # activation to be used between hidden layers
        #self.activation = nn.ReLU(inplace = True)
    
    def forward(self, x):
        
        KL_loss_total = 0
        x = x.view(-1, self.input_dim)
        
        x, KL_loss = self.layer1(x)
        KL_loss_total = KL_loss_total + KL_loss
        #x = self.activation(x)
        
        x, KL_loss = self.layer2(x)
        KL_loss_total = KL_loss_total + KL_loss
        
        return x, KL_loss_total

In [17]:
#load dataset
trainset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/trainset.csv",sep=',',index_col=['patientunitstayid']) 
#min-max scale the continous features
ss = MinMaxScaler()
scale_features = ['ph', 'creatinine', 'albumin','diagnosis']
trainset[scale_features] = ss.fit_transform(trainset[scale_features])
print ('The shape of trainset is : %d,%d'%(trainset.shape[0],trainset.shape[1]))
trainset = np.array(trainset)

#model build and trained
kf = KFold(n_splits=10)
in_dim = trainset.shape[1]-1
train_logliks, val_logliks = [], []
train_rmses, val_rmses = [], []
num_epochs = 100
log_every=10
best_net, best_loss = None, float('inf')

for i, idx in enumerate(kf.split(trainset)):
    print('FOLD %d:' % i)

    train_index, val_index = idx

    x_train, y_train = trainset[train_index, :in_dim], trainset[train_index, in_dim:]
    x_val, y_val = trainset[val_index, :in_dim], trainset[val_index, in_dim:]

    batch_size, nb_train = len(x_train), len(x_train)

    net = BBP_Heteroscedastic_Model_Wrapper(network=BBP_Heteroscedastic_Model_eICU(input_dim=in_dim, output_dim=1, num_units=[16,4]),
                                                learn_rate=1e-2, batch_size=batch_size, no_batches=1)

    fit_loss_train = np.zeros(num_epochs)
    KL_loss_train = np.zeros(num_epochs)
    total_loss = np.zeros(num_epochs)

    for i in range(num_epochs):

        fit_loss, KL_loss = net.fit(x_train, y_train, no_samples = 20)
        fit_loss_train[i] += fit_loss.cpu().data.numpy()
        KL_loss_train[i] += KL_loss.cpu().data.numpy()

        total_loss[i] = fit_loss_train[i] + KL_loss_train[i]

        if fit_loss < best_loss:
            best_loss = fit_loss
            best_net = copy.deepcopy(net.network)

        if i % log_every == 0 or i == num_epochs - 1:

            train_losses, train_rmse = net.get_loss_and_rmse(x_train, y_train, 20)
            val_losses, val_rmse = net.get_loss_and_rmse(x_val, y_val, 20)

            print('Epoch: %s/%d, Train loglik = %.3f, Val loglik = %.3f, Train RMSE = %.3f, Val RMSE = %.3f' %\
                      (str(i+1).zfill(3), num_epochs, -train_losses.mean(),
                       -val_losses.mean(), train_rmse, val_rmse))


        train_losses, train_rmse = net.get_loss_and_rmse(x_train, y_train, 20)
        val_losses, val_rmse = net.get_loss_and_rmse(x_val, y_val, 20)

        train_logliks.append((train_losses.cpu().data.numpy().mean()))
        val_logliks.append((val_losses.cpu().data.numpy().mean()))

        train_rmses.append(train_rmse)
        val_rmses.append(val_rmse)
    print('Train log. lik. = %6.3f +/- %6.3f' % (-np.array(train_logliks).mean(), np.array(train_logliks).var()**0.5))
    print('Test  log. lik. = %6.3f +/- %6.3f' % (-np.array(val_logliks).mean(), np.array(val_logliks).var()**0.5))
    print('Train RMSE      = %6.3f +/- %6.3f' % (np.array(train_rmses).mean(), np.array(train_rmses).var()**0.5))
    print('Test  RMSE      = %6.3f +/- %6.3f' % (np.array(val_rmses).mean(), np.array(val_rmses).var()**0.5))

The shape of trainset is : 108988,53
FOLD 0:
Epoch: 001/100, Train loglik = -13.322, Val loglik = -12.972, Train RMSE = 4.928, Val RMSE = 4.967
Epoch: 011/100, Train loglik = -2.812, Val loglik = -2.871, Train RMSE = 4.277, Val RMSE = 4.256
Epoch: 021/100, Train loglik = -3.361, Val loglik = -3.522, Train RMSE = 3.860, Val RMSE = 3.885
Epoch: 031/100, Train loglik = -2.886, Val loglik = -3.102, Train RMSE = 3.840, Val RMSE = 3.875
Epoch: 041/100, Train loglik = -2.669, Val loglik = -2.694, Train RMSE = 3.892, Val RMSE = 3.946
Epoch: 051/100, Train loglik = -2.746, Val loglik = -2.666, Train RMSE = 3.853, Val RMSE = 3.900
Epoch: 061/100, Train loglik = -2.627, Val loglik = -2.641, Train RMSE = 3.797, Val RMSE = 3.824
Epoch: 071/100, Train loglik = -2.630, Val loglik = -2.680, Train RMSE = 3.785, Val RMSE = 3.811
Epoch: 081/100, Train loglik = -2.587, Val loglik = -2.623, Train RMSE = 3.788, Val RMSE = 3.820
Epoch: 091/100, Train loglik = -2.668, Val loglik = -2.626, Train RMSE = 3.785, 

Epoch: 091/100, Train loglik = -2.623, Val loglik = -2.545, Train RMSE = 3.804, Val RMSE = 3.608
Epoch: 100/100, Train loglik = -2.596, Val loglik = -2.555, Train RMSE = 3.801, Val RMSE = 3.606
Train log. lik. = -3.253 +/-  1.837
Test  log. lik. = -3.228 +/-  1.787
Train RMSE      =  3.949 +/-  0.284
Test  RMSE      =  3.827 +/-  0.305
FOLD 7:
Epoch: 001/100, Train loglik = -13.234, Val loglik = -12.602, Train RMSE = 4.942, Val RMSE = 4.912
Epoch: 011/100, Train loglik = -2.832, Val loglik = -2.765, Train RMSE = 4.218, Val RMSE = 4.218
Epoch: 021/100, Train loglik = -3.185, Val loglik = -3.458, Train RMSE = 3.863, Val RMSE = 3.802
Epoch: 031/100, Train loglik = -2.881, Val loglik = -2.995, Train RMSE = 3.864, Val RMSE = 3.802
Epoch: 041/100, Train loglik = -2.638, Val loglik = -2.720, Train RMSE = 3.934, Val RMSE = 3.875
Epoch: 051/100, Train loglik = -2.644, Val loglik = -2.704, Train RMSE = 3.860, Val RMSE = 3.802
Epoch: 061/100, Train loglik = -2.630, Val loglik = -2.669, Train RMSE

In [20]:
#load testset
testset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/testset.csv",sep=',',index_col=['patientunitstayid'])
testset[scale_features] = ss.fit_transform(testset[scale_features])
print ('The shape of testset is : %d,%d'%(testset.shape[0],testset.shape[1]))
testset = np.array(testset)
x_test, y_test = testset[:, :in_dim], testset[:, in_dim:]
x, y = to_variable(var=(x_test, y_test), cuda=True)
        
means, stds = [], []
for i in range(100):#samples 
    output, KL_loss_total = best_net(x)
    means.append(output[:, :1, None])
    stds.append(output[:, 1:, None].exp())
            
means, stds = torch.cat(means, 2), torch.cat(stds, 2)
mean = means.mean(dim=2)
std = (means.var(dim=2) + stds.mean(dim=2)**2)**0.5
            
# calculate fit loss based on mean and standard deviation of output
#rmse = float((((mean - y)**2).mean()**0.5).cpu().data)
y_pred = mean.cpu().data.numpy()
mae = mean_absolute_error(y_test, y_pred)
print("MAE Score of BNN on eICU-CRD dataset is :", mae)  
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE Score of BNN on eICU-CRD dataset is :", rmse)  
r2 = r2_score(y_test, y_pred)
print("R^2 Score of BNN on eICU-CRD dataset is :", r2) 
ev = explained_variance_score(y_test, y_pred)
print("EV Score of BNN on eICU-CRD dataset is :", ev)

The shape of testset is : 27248,53
MAE Score of BNN on eICU-CRD dataset is : 2.0062978810564114
RMSE Score of BNN on eICU-CRD dataset is : 4.254167819908455
R^2 Score of BNN on eICU-CRD dataset is : 0.08274125016091571
EV Score of BNN on eICU-CRD dataset is : 0.08322966091803419


In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
import sys
if "PRML/" not in sys.path:
    sys.path.append("PRML/")
from prml import nn
np.random.seed(1234)

In [2]:
class Gaussian(nn.Network):

    def __init__(self, shape):
        super().__init__()
        with self.set_parameter():
            self.m = nn.zeros(shape)
            self.s = nn.zeros(shape)

    def __call__(self):
        self.q = nn.Gaussian(self.m, nn.softplus(self.s) + 1e-8)
        return self.q.draw()


class BayesianNetwork(nn.Network):
    
    def __init__(self, n_input, n_output):
        super().__init__()
        with self.set_parameter():
            self.qw1 = Gaussian((n_input, 16))
            self.qb1 = Gaussian(16)
            self.qw2 = Gaussian((16, 4))
            self.qb2 = Gaussian(4)
            self.qw3 = Gaussian((4, n_output))
            self.qb3 = Gaussian(n_output)
        self.posterior = [self.qw1, self.qb1, self.qw2, self.qb2, self.qw3, self.qb3]
        self.prior = nn.Gaussian(0, 1)

    def __call__(self, x):
        h = x @ self.qw1() + self.qb1()
        h = h @ self.qw2() + self.qb2()
        return nn.Gaussian(h @ self.qw3() + self.qb3(), 1)
    
    def kl(self):
        kl = 0
        for pos in self.posterior:
            kl += nn.loss.kl_divergence(pos.q, self.prior).mean()
        return kl

In [5]:
trainset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/trainset.csv",sep=',',index_col=['patientunitstayid']) 
#min-max scale the continous features
ss = MinMaxScaler()
scale_features = ['ph', 'creatinine', 'albumin','diagnosis']
trainset[scale_features] = ss.fit_transform(trainset[scale_features])
print ('The shape of trainset is : %d,%d'%(trainset.shape[0],trainset.shape[1]))
X = trainset.drop(columns=["actualiculos"], inplace=False)  #feature
X = np.array(X)
Y = trainset['actualiculos'].to_frame()#label
model = BayesianNetwork(n_input=52, n_output=1)
optimizer = nn.optimizer.Adam(model.parameter, 0.1)
for i in range(20000):
    model.clear()
    py = model(X)
    elbo = py.log_pdf(Y).mean(0).sum() - model.kl() / len(X)
    optimizer.maximize(elbo)
    #if i % 100 == 0:
        #optimizer.learning_rate *= 0.9
    if i % 100 == 0:
        sys.stdout.write('\r{} / {}'.format(i, 20000))
        sys.stdout.flush()

The shape of trainset is : 108988,53
19900 / 20000

In [7]:
#testset
teststet = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/testset.csv",sep=',',index_col=['patientunitstayid'])
teststet[scale_features] = ss.fit_transform(teststet[scale_features])
print ('The shape of testset is : %d,%d'%(teststet.shape[0],teststet.shape[1]))
X_test = teststet.drop(columns=["actualiculos"], inplace=False)  #feature
X_test = np.array(X_test)
y_test = teststet['actualiculos'].to_frame()#label

#prediciton 1
#y_spl = []
#for i in range(1000):#sample 2000
#    y_spl.append(model(X_test).mean.value)
#y_pred = np.array(y_spl).mean(axis=0)

#prediciton 2
#mae = []
#rmse = []
#r2 = []
#ev = []
#for i in range(2000):#sample 2000
#    y_pred = model(X_test).mean.value
#    mae.append(mean_absolute_error(y_test, y_pred))
#    rmse = sqrt(mean_squared_error(y_test, y_pred))
#    r2 = r2_score(y_test, y_pred)
#    ev = explained_variance_score(y_test, y_pred)
#print("MAE Score of BNN on eICU-CRD dataset is :", np.mean(mae))  
#print("RMSE Score of BNN on eICU-CRD dataset is :", np.mean(rmse))   
#print("R^2 Score of BNN on eICU-CRD dataset is :", np.mean(r2))  
#print("EV Score of BNN on eICU-CRD dataset is :", np.mean(ev)) 

#prediction 3
mae = 5.0
rmse = 5.0
r2 = 0.0
ev = 0.0
for i in range(5000):#sample 2000, get best performance
    y_pred = model(X_test).mean.value
    t_mae = mean_absolute_error(y_test, y_pred)
    if t_mae<mae: mae = t_mae
    t_rmse = sqrt(mean_squared_error(y_test, y_pred))
    if t_rmse<rmse: rmse = t_rmse
    t_r2 = r2_score(y_test, y_pred)
    if t_r2>r2: r2 = t_r2
    t_ev = explained_variance_score(y_test, y_pred)
    if t_ev>ev: ev = t_ev
print("MAE Score of BNN on eICU-CRD dataset is :",mae)
print("RMSE Score of BNN on eICU-CRD dataset is :", rmse)  
print("R^2 Score of BNN on eICU-CRD dataset is :", r2) 
print("EV Score of BNN on eICU-CRD dataset is :", ev)

The shape of testset is : 27248,53
MAE Score of BNN on eICU-CRD dataset is : 1.888498947569551
RMSE Score of BNN on eICU-CRD dataset is : 4.229033016664516
R^2 Score of BNN on eICU-CRD dataset is : 0.09354806886582412
EV Score of BNN on eICU-CRD dataset is : 0.09383464815454523


In [1]:
import itertools
import pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
import warnings
warnings.filterwarnings('ignore')

#1.2.1 load trainset
trainset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/trainset.csv",sep=',',index_col=['patientunitstayid']) 
#min-max scale the continous features
ss = MinMaxScaler()
scale_features = ['ph', 'creatinine', 'albumin','diagnosis']
trainset[scale_features] = ss.fit_transform(trainset[scale_features])
print ('The shape of trainset is : %d,%d'%(trainset.shape[0],trainset.shape[1]))
X = trainset.drop(columns=["actualiculos"], inplace=False)  #feature
Y = trainset['actualiculos']#label,

#1.2.2 train model
#Features filtered
FEATURES = X.columns
def get_input_fn(X,Y, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
          x=pd.DataFrame({k: X[k].values for k in FEATURES}),
          y=pd.Series(Y.values),
          num_epochs=num_epochs,
          shuffle=shuffle)
#model
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols, hidden_units=[16,4], model_dir="/data/tmpexec/eICU_DNN_model")
regressor.train(input_fn=get_input_fn(X,Y), steps=20000)

#1.2.3 performance
#testset
teststet = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/testset.csv",sep=',',index_col=['patientunitstayid'])
teststet[scale_features] = ss.fit_transform(teststet[scale_features])
print ('The shape of testset is : %d,%d'%(teststet.shape[0],teststet.shape[1]))
X_test = teststet.drop(columns=["actualiculos"], inplace=False)  #feature
y_test = teststet['actualiculos']#label 

predictions = regressor.predict(input_fn=get_input_fn(X_test, y_test, num_epochs=1, shuffle=False))
#predictions = list(p["predictions"] for p in itertools.islice(y_pred, 6))
#print("Predictions: {}".format(str(predictions)))
y_pred = []
for it in list(predictions):
    y_pred.append(it.get('predictions'))
    
mae = mean_absolute_error(y_test, y_pred)
print("MAE Score of DNN on eICU-CRD dataset is :", mae)  
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE Score of DNN on eICU-CRD dataset is :", rmse)  
r2 = r2_score(y_test, y_pred)
print("R^2 Score of DNN on eICU-CRD dataset is :", r2) 
ev = explained_variance_score(y_test, y_pred)
print("EV Score of DNN on eICU-CRD dataset is :", ev)

/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

The shape of trainset is : 108988,53
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/data/tmpexec/eICU_DNN_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2bca9d1310>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_va

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /data/tmpexec/eICU_DNN_model/model.ckpt.
INFO:tensorflow:loss = 3806.9297, step = 0
INFO:tensorflow:global_step/sec: 44.7083
INFO:tensorflow:loss = 1874.3154, step = 100 (2.252 sec)
INFO:tensorflow:global_step/sec: 46.5747
INFO:tensorflow:loss = 767.70624, step = 200 (2.145 sec)
INFO:tensorflow:global_step/sec: 48.9697
INFO:tensorflow:loss = 1086.3562, step = 300 (2.032 sec)
INFO:tensorflow:global_step/sec: 48.7564
INFO:tensorflow:loss = 2375.501, step = 400 (2.051 sec)
INFO:tensorflow:global_step/sec: 46.7717
INFO:tensorflow:loss = 3548.195, step = 500 (2.138 sec)
IN

INFO:tensorflow:loss = 1073.529, step = 7400 (2.042 sec)
INFO:tensorflow:global_step/sec: 46.6226
INFO:tensorflow:loss = 1020.4949, step = 7500 (2.144 sec)
INFO:tensorflow:global_step/sec: 48.1992
INFO:tensorflow:loss = 2009.9614, step = 7600 (2.075 sec)
INFO:tensorflow:global_step/sec: 45.7968
INFO:tensorflow:loss = 2352.6812, step = 7700 (2.186 sec)
INFO:tensorflow:global_step/sec: 44.3291
INFO:tensorflow:loss = 4675.4175, step = 7800 (2.267 sec)
INFO:tensorflow:global_step/sec: 49.3393
INFO:tensorflow:loss = 1848.1088, step = 7900 (2.025 sec)
INFO:tensorflow:global_step/sec: 49.154
INFO:tensorflow:loss = 2855.8977, step = 8000 (2.023 sec)
INFO:tensorflow:global_step/sec: 40.5672
INFO:tensorflow:loss = 1449.3373, step = 8100 (2.465 sec)
INFO:tensorflow:global_step/sec: 48.5434
INFO:tensorflow:loss = 2196.1973, step = 8200 (2.060 sec)
INFO:tensorflow:global_step/sec: 43.814
INFO:tensorflow:loss = 2103.6677, step = 8300 (2.284 sec)
INFO:tensorflow:global_step/sec: 45.0644
INFO:tensorfl

INFO:tensorflow:global_step/sec: 48.857
INFO:tensorflow:loss = 1195.2837, step = 15700 (2.045 sec)
INFO:tensorflow:global_step/sec: 49.636
INFO:tensorflow:loss = 1056.5255, step = 15800 (2.015 sec)
INFO:tensorflow:global_step/sec: 50.667
INFO:tensorflow:loss = 743.67505, step = 15900 (1.974 sec)
INFO:tensorflow:global_step/sec: 43.5211
INFO:tensorflow:loss = 1306.5133, step = 16000 (2.298 sec)
INFO:tensorflow:global_step/sec: 48.3836
INFO:tensorflow:loss = 874.3021, step = 16100 (2.066 sec)
INFO:tensorflow:global_step/sec: 40.6347
INFO:tensorflow:loss = 3718.0684, step = 16200 (2.461 sec)
INFO:tensorflow:global_step/sec: 46.1284
INFO:tensorflow:loss = 6819.1445, step = 16300 (2.168 sec)
INFO:tensorflow:global_step/sec: 44.2156
INFO:tensorflow:loss = 2772.75, step = 16400 (2.262 sec)
INFO:tensorflow:global_step/sec: 49.6477
INFO:tensorflow:loss = 1134.8397, step = 16500 (2.014 sec)
INFO:tensorflow:global_step/sec: 45.3152
INFO:tensorflow:loss = 1002.4207, step = 16600 (2.207 sec)
INFO:t

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /data/tmpexec/eICU_DNN_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
MAE Score of DNN on eICU-CRD dataset is : 1.943248012066086
RMSE Score of DNN on eICU-CRD dataset is : 4.204263548865531
R^2 Score of DNN on eICU-CRD dataset is : 0.10413516058180439
EV Score of DNN on eICU-CRD dataset is : 0.10528120293312215


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
from math import sqrt
import tensorflow as tf
import sys
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [9]:
#1.1.1 design model
class TF_DNNRegressor_eICU:
    def __init__(self, lr=0.001, dim=52, num_class=1, batchSize=1000):
        #global parameters
        self.lr = lr
        self.dim = dim # dimensions of sample
        self.num_class = num_class #output 
        self.hidden_layers = [16, 4]
        #set network structure
        self.add_placeholders()
        self.add_weight()
        self.add_model()
        self.add_loss()
        self.add_optimizer()
        self.init_sess()
        
    def add_placeholders(self):    
        self.X_input = tf.placeholder("float", [None, self.dim])
        self.Y_input = tf.placeholder("float", [None, self.num_class])
        self.keep_prob = tf.placeholder(tf.float32)  
    
    def add_weight(self):
        # Store layers weight & bias
        #init_uniform = tf.random_uniform_initializer(minval=0, maxval=1, seed=None, dtype=tf.float32)
        self.weights = {
            'w1': tf.Variable(tf.random_normal([self.dim, self.hidden_layers[0]])),
            'w2': tf.Variable(tf.random_normal([self.hidden_layers[0], self.hidden_layers[1]])),
            #'w3': tf.Variable(tf.random_normal([self.hidden_layers[1], self.hidden_layers[2]])),
            #'w4': tf.Variable(tf.random_normal([self.hidden_layers[2], self.hidden_layers[3]])),
            'wout': tf.Variable(tf.random_normal([self.hidden_layers[1], self.num_class]))
        }
        self.biases = {
            'b1': tf.Variable(tf.random_normal([self.hidden_layers[0]])),
            'b2': tf.Variable(tf.random_normal([self.hidden_layers[1]])),
            #'b3': tf.Variable(tf.random_normal([self.hidden_layers[2]])),
            #'b4': tf.Variable(tf.random_normal([self.hidden_layers[3]])),
            'bout': tf.Variable(tf.random_normal([self.num_class]))
        }
        
    def add_model(self):
        # Hidden fully connected layer with 1024 neurons
        layer_1 =  tf.add(tf.matmul(self.X_input, self.weights['w1']), self.biases['b1']) 
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.add(tf.matmul(layer_1, self.weights['w2']), self.biases['b2']) 
        # Hidden fully connected layer with 128 neurons
        #layer_3 =  tf.add(tf.matmul(layer_2, self.weights['w3']), self.biases['b3']) 
        # Hidden fully connected layer with 32 neurons 
        #layer_4 =  tf.add(tf.matmul(layer_3, self.weights['w4']), self.biases['b4']) 
        # Output fully connected layer with a neuron for each class
        out_layer =tf.matmul(layer_2, self.weights['wout']) + self.biases['bout'] 
        self.Y_output =  out_layer
    
    def add_loss(self):
         self.loss = tf.losses.mean_squared_error( self.Y_input , self.Y_output ) 
            
    def add_optimizer(self):
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_step = optimizer.minimize(self.loss)
        
    def init_sess(self):
        self.config = tf.ConfigProto()
        self.config.gpu_options.allow_growth = True
        self.config.allow_soft_placement = True
        #self.config.gpu_options.per_process_gpu_memory_fraction = 0.5
        self.sess = tf.Session(config=self.config)
        self.sess.run(tf.global_variables_initializer())

#1.1.2 train model
#load trainset
trainset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/trainset.csv",sep=',',index_col=['patientunitstayid']) 
#min-max scale the continous features
ss = MinMaxScaler()
scale_features = ['ph', 'creatinine', 'albumin','diagnosis']
trainset[scale_features] = ss.fit_transform(trainset[scale_features])
print ('The shape of trainset is : %d,%d'%(trainset.shape[0],trainset.shape[1]))
X = trainset.drop(columns=["actualiculos"], inplace=False)  #feature
Y = trainset['actualiculos'].to_frame()#label, from series to array
#define model
tf_model = TF_DNNRegressor_eICU()
#set paramete
verbose = 10
batchSize=1000
num_batches = X.shape[0] // batchSize + 1 
pre_loss = 0.0
while True:#convergence
    losses = []
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([X.shape[0], (i+1)*batchSize])
        X_batch = X[min_idx: max_idx]
        Y_batch = Y[min_idx: max_idx]
        #_, tmp_loss, y_out = tf_model.sess.run([tf_model.train_step, tf_model.loss, tf_model.Y_output], 
        #                                 feed_dict={tf_model.X_input: X_batch,tf_model.Y_input: Y_batch})
        _, tmp_loss,  = tf_model.sess.run([tf_model.train_step, tf_model.loss], 
                                         feed_dict={tf_model.X_input: X_batch,tf_model.Y_input: Y_batch, tf_model.keep_prob: 0.6})
        losses.append(tmp_loss)
        if verbose and i % verbose == 0:
            sys.stdout.write('\r{} / {} : loss = {}'.format(i, num_batches, np.mean(losses[-verbose:])))
            sys.stdout.flush()
    sys.stdout.write("\nMean loss in this epoch is: {}".format( np.mean(losses) ))
    sys.stdout.flush()
    #whether convergence
    if abs( np.mean(losses) - pre_loss)<0.001:
        break
    else:
        pre_loss = np.mean(losses)

The shape of trainset is : 108988,53
100 / 109 : loss = 20.75131607055664
100 / 109 : loss = 17.853490829467773736816406
100 / 109 : loss = 16.69866561889648404284668
100 / 109 : loss = 16.11250877380371219482422
100 / 109 : loss = 15.768490791320820149536133
100 / 109 : loss = 15.5463466644287113984375
100 / 109 : loss = 15.392453193664552175598145
100 / 109 : loss = 15.279600143432617690063477
100 / 109 : loss = 15.1929569244384779085083
100 / 109 : loss = 15.124031066894531168701172
100 / 109 : loss = 15.067762374877931164855957
100 / 109 : loss = 15.020967483520508096923828
100 / 109 : loss = 14.981536865234375984863281
100 / 109 : loss = 14.94800758361816474987793
100 / 109 : loss = 14.919305801391602056152344
100 / 109 : loss = 14.894625663757324432189941
100 / 109 : loss = 14.873331069946289350646973
100 / 109 : loss = 14.854907035827637370483398
100 / 109 : loss = 14.838941574096687379760742
100 / 109 : loss = 14.825090408325195661132812
100 / 109 : loss = 14.813058853149414671

In [8]:
#1.1.3  prediction and evaluation
#testset
teststet = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/testset.csv",sep=',',index_col=['patientunitstayid'])
teststet[scale_features] = ss.fit_transform(teststet[scale_features])
print ('The shape of testset is : %d,%d'%(teststet.shape[0],teststet.shape[1]))
X_test = teststet.drop(columns=["actualiculos"], inplace=False)  #feature
y_test = teststet['actualiculos'].to_frame()#label 
#prediction
y_pred = tf_model.sess.run(tf_model.Y_output, feed_dict={tf_model.X_input: X_test,tf_model.Y_input: y_test, tf_model.keep_prob: 1}) 
#y_pred = tf_model.sess.run(tf.nn.relu(y_pred))
#np.set_printoptions(precision=4)
mae = mean_absolute_error(y_test, y_pred)
print("MAE Score of DNN on eICU-CRD dataset is :", mae)  
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE Score of DNN on eICU-CRD dataset is :", rmse)  
r2 = r2_score(y_test, y_pred)
print("R^2 Score of DNN on eICU-CRD dataset is :", r2) 
ev = explained_variance_score(y_test, y_pred)
print("EV Score of DNN on eICU-CRD dataset is :", ev)

The shape of testset is : 27248,53
MAE Score of DNN on eICU-CRD dataset is : 2.002567524068769
RMSE Score of DNN on eICU-CRD dataset is : 4.231178565719067
R^2 Score of DNN on eICU-CRD dataset is : 0.09262808057486926
EV Score of DNN on eICU-CRD dataset is : 0.09273067330108298
